# RL -- Projet "Trading automatique"

Ce notebook contient du code de base et quelques explications pour vous aider sur ce sujet.

Vous êtes libres de réaliser ce projet avec des scripts Python ou des Jupyter Notebooks, à votre convenance.

Vous devez télécharger les paquets Python suivants :

```sh
pip install gymnasium
pip install pandas
pip install gym-trading-env-continuous
```

Vous utiliserez l'environnement `gym-trading-env-continuous`, qui est un *fork* de [Gym Trading Env](https://gym-trading-env.readthedocs.io/en/latest/index.html). La différence majeure est expliquée dans ce document ; la documentation originelle reste utilisable.

In [2]:
!pip install gymnasium
!pip install pandas
!pip install gym-trading-env-continuous

In [3]:
import numpy as np
import pandas as pd
import gymnasium as gym
import gym_trading_env
import os
import ta # Assurez-vous d'installer la librairie : pip install ta (Pour l'ajout de nouvelle feature)


## Utilisation des données de simulation

Les données sont dans un format binaire (Pickle) que vous pouvez lire avec Pandas. Vous devez vous assurer que les données sont triées par date.

Des étapes de prétraitement peuvent aider votre apprentissage, par exemple, supprimer les doublons, etc.

In [ ]:
def preprocess(df):
    df = df.sort_index()
    df = df.dropna()
    df = df.drop_duplicates()
    return df

# Lister tous les fichiers et dossiers dans le répertoire
for filename in os.listdir('data'):
    # Construire le chemin complet du fichier
    filepath = os.path.join('data', filename)

    # Vérifier si c'est un fichier et qu'il se termine par '.pkl'
    if os.path.isfile(filepath) and filename.endswith('.pkl'):
        print(f"\n--- Aperçu de '{filename}' 5 first elements")
        df = preprocess(pd.read_pickle(filepath))
        display(df.head(5))
        




--- Aperçu de 'binance-BTCUSD-1h.pkl' 5 first elements


,open,high,low,close,volume,date_close
date_open,,,,,,
2020-08-11 07:00:00,11785.0,11785.0,11735.0,11751.4,7.560836,2020-08-11 08:00:00
2020-08-11 08:00:00,11752.2,11780.2,11711.9,11717.7,191.028616,2020-08-11 09:00:00
2020-08-11 09:00:00,11718.9,11772.3,11704.2,11716.2,189.694611,2020-08-11 10:00:00
2020-08-11 10:00:00,11708.9,11762.1,11656.3,11749.5,199.613807,2020-08-11 11:00:00
2020-08-11 11:00:00,11750.2,11769.9,11725.0,11752.2,157.709769,2020-08-11 12:00:00



--- Aperçu de 'binance-DOGEEUR-1h.pkl' 5 first elements


,open,high,low,close,volume,date_close
date_open,,,,,,
2021-01-29 12:00:00,0.04684,0.19000,0.03900,0.05181,51358749.0,2021-01-29 13:00:00
2021-01-29 13:00:00,0.05181,0.05699,0.04200,0.04413,47022463.7,2021-01-29 14:00:00
2021-01-29 14:00:00,0.04423,0.04773,0.03853,0.04429,34717160.7,2021-01-29 15:00:00
2021-01-29 15:00:00,0.04429,0.04995,0.04399,0.04548,25380511.1,2021-01-29 16:00:00
2021-01-29 16:00:00,0.04548,0.04770,0.04000,0.04356,28716787.9,2021-01-29 17:00:00



--- Aperçu de 'binance-ETHUSD-1h.pkl' 5 first elements


...

,open,high,low,close,volume,date_close
date_open,,,,,,
2023-11-21 13:00:00,1991.000000,2002.699951,1991.000000,1999.599976,0,2023-11-21 14:00:00
2023-11-21 14:00:00,1999.699951,2009.699951,1999.300049,2007.599976,36358,2023-11-21 15:00:00
2023-11-21 15:00:00,2007.699951,2009.800049,2005.199951,2006.500000,25776,2023-11-21 16:00:00
2023-11-21 16:00:00,2006.500000,2007.599976,2000.900024,2003.300049,17914,2023-11-21 17:00:00
2023-11-21 17:00:00,2003.400024,2003.800049,2001.199951,2003.699951,9145,2023-11-21 18:00:00



--- Aperçu de 'yfinance-S&P500-1h.pkl' 5 first elements


,open,high,low,close,volume,date_close
date_open,,,,,,
2023-11-21 14:30:00,4538.770020,4538.770020,4526.350098,4526.959961,0,2023-11-21 15:30:00
2023-11-21 15:30:00,4527.009766,4533.509766,4525.509766,4531.029785,257387382,2023-11-21 16:30:00
2023-11-21 16:30:00,4531.100098,4537.859863,4528.629883,4537.669922,192374214,2023-11-21 17:30:00
2023-11-21 17:30:00,4537.660156,4541.899902,4532.899902,4534.600098,162898033,2023-11-21 18:30:00
2023-11-21 18:30:00,4534.580078,4541.750000,4534.200195,4538.600098,174924000,2023-11-21 19:30:00



--- Aperçu de 'yfinance-STOXX50-1h.pkl' 5 first elements


,open,high,low,close,volume,date_close
date_open,,,,,,
2023-11-21 13:00:00,4334.049805,4334.049805,4329.779785,4331.120117,0,2023-11-21 14:00:00
2023-11-21 14:00:00,4331.020020,4339.259766,4331.020020,4335.509766,0,2023-11-21 15:00:00
2023-11-21 15:00:00,4334.319824,4337.430176,4330.270020,4333.229980,0,2023-11-21 16:00:00
2023-11-21 16:00:00,4333.080078,4333.200195,4329.529785,4332.600098,0,2023-11-21 17:00:00
2023-11-22 08:00:00,4335.779785,4347.240234,4335.779785,4347.240234,0,2023-11-22 09:00:00


### Ajout de *features*

Vous pouvez également rajouter de nouvelles données au DataFrame pour créer de nouvelles *features* que l'agent pourra utiliser.
Voir pour cela la [doc](https://gym-trading-env.readthedocs.io/en/latest/features.html).

Chaque nouvelle *feature* doit commencer par `feature_` pour être détectée.

In [9]:
def preprocess(df):
    df = df.sort_index()
    df = df.dropna()
    df = df.drop_duplicates()

    df['feature_close'] = (df['close'] - df['close'].mean()) / df['close'].std()

    # Ajout de nouvelles Features
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
    df['MACD'] = ta.trend.MACD(df['close']).macd()
    df['BB_High'] = ta.volatility.BollingerBands(df['close']).bollinger_hband()
    df['MA_20'] = df['close'].rolling(window=20).mean()
    df['volume_norm'] = df['volume'] / df['volume'].rolling(window=50).mean()


    df = df.dropna()
    
    return df


for filename in os.listdir('data'):
    filepath = os.path.join('data', filename)

    if os.path.isfile(filepath) and filename.endswith('.pkl'):
        df = preprocess(pd.read_pickle(filepath))
        print(f"\n--- Aperçu de '{filename}' 5 first elements")
        display(df.head(5))





--- Aperçu de 'binance-BTCUSD-1h.pkl' 5 first elements


,open,high,low,close,volume,date_close,feature_close,RSI,MACD,BB_High,MA_20,volume_norm
date_open,,,,,,,,,,,,
2020-08-13 08:00:00,11528.6,11532.0,11380.6,11431.8,324.334622,2020-08-13 09:00:00,-1.308690,40.415607,5.007262,11650.236610,11563.195,1.329401
2020-08-13 09:00:00,11428.7,11436.0,11278.9,11409.5,473.239028,2020-08-13 10:00:00,-1.309433,38.815403,-5.826905,11664.906995,11555.195,1.868412
2020-08-13 10:00:00,11409.8,11480.7,11355.4,11480.7,286.726970,2020-08-13 11:00:00,-1.307061,46.146964,-8.569037,11661.279332,11549.155,1.123547
2020-08-13 11:00:00,11481.0,11631.6,11448.6,11561.8,316.926461,2020-08-13 12:00:00,-1.304359,53.048274,-4.150262,11658.667239,11547.820,1.229624
2020-08-13 12:00:00,11561.1,11594.1,11392.7,11473.2,340.205813,2020-08-13 13:00:00,-1.307311,46.098013,-7.708769,11658.978021,11543.605,1.305699



--- Aperçu de 'binance-DOGEEUR-1h.pkl' 5 first elements


,open,high,low,close,volume,date_close,feature_close,RSI,MACD,BB_High,MA_20,volume_norm
date_open,,,,,,,,,,,,
2021-01-31 13:00:00,0.02500,0.02531,0.02375,0.02399,12031712.9,2021-01-31 14:00:00,-1.359242,44.677303,-0.001484,0.025815,0.023333,0.601842
2021-01-31 14:00:00,0.02400,0.02466,0.02356,0.02415,7135697.5,2021-01-31 15:00:00,-1.357322,45.278909,-0.001353,0.025873,0.023437,0.373459
2021-01-31 15:00:00,0.02415,0.02444,0.02242,0.02373,9180220.0,2021-01-31 16:00:00,-1.362361,43.928488,-0.001268,0.025615,0.023601,0.500279
2021-01-31 16:00:00,0.02377,0.02576,0.02344,0.02458,19843135.7,2021-01-31 17:00:00,-1.352164,47.350799,-0.001120,0.025449,0.023779,1.099178
2021-01-31 17:00:00,0.02458,0.02897,0.02451,0.02818,27588559.1,2021-01-31 18:00:00,-1.308974,58.815840,-0.000703,0.026491,0.024077,1.524493



--- Aperçu de 'binance-ETHUSD-1h.pkl' 5 first elements


...

,open,high,low,close,volume,date_close,feature_close,RSI,MACD,BB_High,MA_20,volume_norm
date_open,,,,,,,,,,,,
2023-11-23 16:00:00,1993.099976,1993.199951,1990.900024,1991.599976,5926,2023-11-23 17:00:00,-1.388399,38.607810,-1.218276,1998.627796,1994.449988,0.791734
2023-11-23 17:00:00,1991.599976,1994.000000,1991.599976,1993.800049,1596,2023-11-23 18:00:00,-1.384704,45.736720,-1.153502,1998.543162,1994.559991,0.212326
2023-11-23 18:00:00,1993.900024,1994.000000,1992.699951,1992.800049,436,2023-11-23 19:00:00,-1.386383,43.276845,-1.169379,1998.445402,1994.624994,0.064133
2023-11-23 19:00:00,1992.800049,1993.699951,1992.500000,1993.300049,239,2023-11-23 20:00:00,-1.385543,44.873329,-1.128606,1998.315148,1994.714996,0.038011
2023-11-23 23:00:00,1993.699951,1995.099976,1993.300049,1994.800049,810,2023-11-24 00:00:00,-1.383024,49.468215,-0.964142,1998.306291,1994.814996,0.136237



--- Aperçu de 'yfinance-S&P500-1h.pkl' 5 first elements


,open,high,low,close,volume,date_close,feature_close,RSI,MACD,BB_High,MA_20,volume_norm
date_open,,,,,,,,,,,,
2023-12-01 18:30:00,4588.540039,4591.390137,4585.180176,4590.290039,228248000,2023-12-01 19:30:00,-1.921555,65.758217,8.252402,4593.207782,4561.232935,1.120083
2023-12-01 19:30:00,4590.310059,4594.220215,4587.500000,4593.350098,256669000,2023-12-01 20:30:00,-1.916271,66.868865,9.491714,4597.881696,4563.185938,1.228604
2023-12-01 20:30:00,4593.419922,4596.209961,4589.279785,4593.720215,397618000,2023-12-01 21:30:00,-1.915632,67.008251,10.384043,4601.507948,4564.403955,1.878075
2023-12-04 14:30:00,4564.370117,4571.839844,4549.589844,4551.379883,0,2023-12-04 15:30:00,-1.988737,44.133670,7.587248,4601.537498,4564.155957,0.000000
2023-12-04 15:30:00,4551.279785,4554.350098,4546.720215,4553.580078,307065765,2023-12-04 16:30:00,-1.984938,45.180915,5.485076,4601.135470,4563.496973,1.457003



--- Aperçu de 'yfinance-STOXX50-1h.pkl' 5 first elements


,open,high,low,close,volume,date_close,feature_close,RSI,MACD,BB_High,MA_20,volume_norm
date_open,,,,,,,,,,,,


Par défaut, l'agent ne reçoit comme *features* que sa dernière *position* (voir le paragraphe suivant), ce qui ne sera certainement pas suffisant ! À vous d'ajouter les *features* qui seront pertinentes pour que l'agent apprenne la politique optimale...

## Fonctionnement des actions

Une action est une **position**, c'est-à-dire un ratio entre la proportion d'*assets* (exemple : ETH) et la proportion de *fiat* (exemple : USD) dans le portefeuille.
Ainsi, la position `0.5` consiste à avoir exactement 50% d'ETH et 50% d'USD (en vendant l'un ou l'autre pour arriver à ce ratio). `0.1` consiste à avoir 10% d'ETH et 90% d'USD.

Il existe des positions un peu plus complexes :

- `< 0` : une position inférieure à 0 va vendre encore plus d'ETH que le portefeuille n'en contient, pour obtenir des USD. Cela nécessite un emprunt, qui sera remboursé avec un intérêt.
- `> 1` : une position supérieure à 1 va dépenser encore plus d'USD que le portefeuille n'en contient, pour acheter des ETH. Cela nécessite également un emprunt.

Ces positions (qui sont appelées *short* et *margin* en finance) peuvent faire gagner beaucoup à votre agent, mais démultiplient les risques également. Si votre agent fait une bonne affaire, vous pouvez vendre à un prix élevé, racheter quand le prix est plus faible, et rembourser l'emprunt en empochant la différence. En revanche, si votre agent fait une mauvaise affaire, et doit vider son portefeuille pour rembourser l'emprunt, vous perdez automatiquement (`terminated=True`).

### Actions continues

Par rapport à l'environnement `gym-trading-env` d'origine, la version que je vous fournis permet de spécifier directement une position comme action, c'est-à-dire un nombre flottant. Votre agent a donc un contrôle précis sur la position désirée. Cela rajoute de la flexibilité mais rend l'apprentissage beaucoup plus difficile.

Exemple :

In [5]:
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

obs, _ = env.reset()
# On veut une position de 88% ETH / 12% USD
obs, reward, terminated, truncated, info = env.step(0.88)
print(obs)
print(info)

[-1.8911283  0.88       0.8801226]
{'idx': 1, 'step': 1, 'date': np.datetime64('2020-08-18T08:00:00.000000000'), 'position_index': None, 'position': 0.88, 'real_position': np.float64(0.8801225797464827), 'data_open': 430.27, 'data_date_close': Timestamp('2020-08-18 09:00:00'), 'data_high': 431.79, 'data_low': 430.27, 'data_close': 430.8, 'data_volume': 454.17615317, 'portfolio_valuation': np.float64(1000.3680954092978), 'portfolio_distribution_asset': np.float64(2.0437477920791705), 'portfolio_distribution_fiat': np.float64(119.92154658159109), 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0.0, 'portfolio_distribution_interest_fiat': 0.0, 'reward': np.float64(0.00036802767880289296)}


Par défaut, l'espace des actions est limité à $[-1, 2]$ pour que votre agent ne puisse emprunter que jusqu'à 100%. Vous pouvez empêcher votre agent de prendre de telles positions, ou limiter le risque, en contrôlant les bornes autorisées des actions.

Par exemple, en clippant l'action dans l'intervalle $[0,1]$, vous empêchez l'agent de faire des emprunts.

À l'inverse, vous pouvez augmenter l'intervalle pour permettre des emprunts plus risqués, mais qui peuvent rapporter plus. À vous de choisir !

Vous pouvez changer les bornes via le paramètre `position_range` du constructeur :

In [6]:
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    position_range=(0, 1),  # ICI : (borne min, borne max)
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

Vous pouvez aussi modifier l'action en sortie de votre algorithme d'apprentissage, de la manière que vous souhaitez (clipping, interpolation, etc.).

### Actions discrètes

Pour simplifier l'apprentissage, vous pouvez utiliser le *wrapper* `gym_trading_env.wrapper.DiscreteActionsWrapper` que je vous fournis, et qui permet de revenir au fonctionnement d'origine de l'environnement `gym-trading-env`. Vous devrez alors spécifier l'ensemble des positions possibles, puis votre agent choisira une position parmi cette liste à chaque pas de temps.
Par exemple, si la liste des positions est `[0, 0.5, 1]` et que l'action choisie est `1`, cela veut dire qu'on veut la position qui correspond au 2e élément de la liste, soit `0.5` (50%/50%).

Vous pouvez rajouter autant d'actions que vous voulez, par exemple `[0, 0.25, 0.5, 1]` ou encore tous les 0.1 entre 0 et 1, etc. Plus il y a d'actions possibles, plus votre agent aura de choix (flexibilité), donc plus son comportement pourra être complexe, mais cela rajoute de la difficulté durant l'entraînement.

N'oubliez pas que vous pouvez autoriser les positions avec emprunt en ajoutant des nombres inférieurs à 0 ou supérieurs à 1 à la liste autorisée.

Exemple :

In [7]:
from gym_trading_env.wrapper import DiscreteActionsWrapper

# Vous pouvez aussi appeler le wrapper `env` pour faire plus simple
# Ici, je fais explicitement la distinction entre `wrapper` et `env`
wrapper = DiscreteActionsWrapper(env, positions=[-1, 0, 0.25, 0.5, 0.75, 1, 2])
obs, _ = wrapper.reset()
# On veut une position de 25% ETH / 75% USD ; cela correspond à la position
# d'index 2 dans la liste ci-dessus
obs, reward, terminated, truncated, info = wrapper.step(2)
print(obs)
print(info)

[-0.16626613  0.25        0.25014377]
{'idx': 1, 'step': 1, 'date': np.datetime64('2023-11-21T14:00:00.000000000'), 'position_index': 2, 'position': 0.25, 'real_position': np.float64(0.2501437786412375), 'data_open': 1.0949305295944214, 'data_date_close': Timestamp('2023-11-21 15:00:00'), 'data_high': 1.0961307287216187, 'data_low': 1.0948106050491333, 'data_close': 1.0955302715301514, 'data_volume': 0, 'portfolio_valuation': np.float64(999.9537158331323), 'portfolio_distribution_asset': np.float64(228.32066574982062), 'portfolio_distribution_fiat': np.float64(749.8215148882864), 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0.0, 'portfolio_distribution_interest_fiat': 0.0, 'reward': np.float64(-4.6285238012796695e-05)}


Notez que, quand les actions continues sont utilisées, la variable `position_index` du dictionnaire `info` n'est pas disponible (c'est logique).

## Changement de la fonction de récompense

Vous pouvez changer la fonction de récompense pour améliorer l'apprentissage de l'agent.
Dans tous les cas, vous serez évalué(e)s sur la valuation du portefeuille à la fin de l'épisode (voir [ci-dessous](#évaluation)), mais cette simple mesure n'est peut-être pas la meilleure fonction de récompense.
D'autres fonctions peuvent encourager l'agent à mieux apprendre, en explorant diverses possibilités, etc.

In [8]:
def reward_function(history):
    return history['portfolio_valuation', -1]

env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    # On spécifie la fonction de récompense
    reward_function=reward_function,
)

## Déroulément d'un épisode

Un épisode se déroule jusqu'à ce que :

- l'agent atteigne la fin des données d'entraînement (nous n'avons plus de nouvelle donnée) => `truncated=True`

- la valeur du portefeuille atteint 0 (l'agent a perdu tout l'argent) => `terminated=True`

Vous devrez probablement entraîner l'agent sur plusieurs épisodes avant que son comportement ne converge.

Pour éviter de sur-apprendre (*overfit*), vous devrez utiliser plusieurs jeux de données via [MultiDatasetTradingEnv](https://gym-trading-env.readthedocs.io/en/latest/multi_datasets.html).

Dans ce cas, chaque épisode utilisera un jeu de données différent (en bouclant si vous demandez plus d'épisodes qu'il n'y a de jeux de données). Vous pouvez accéder au nom du jeu de données de l'épisode en cours via `env.name`.

In [9]:
nb_episodes = 2
for episode in range(1, nb_episodes + 1):
    obs, _ = env.reset()
    print(f'Episode n˚{episode} -- Jeu de donnée {env.name}')
    done = False

    while not done:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    if terminated:
        print('Argent perdu')
    elif truncated:
        print('Épisode terminé')


Episode n˚1 -- Jeu de donnée yfinance-S&P500-1h.pkl
Market Return : 44.45%   |   Portfolio Return : -96.19%   |   
Épisode terminé
Episode n˚2 -- Jeu de donnée binance-ETHUSD-1h.pkl
Market Return : 904.05%   |   Portfolio Return : -100.00%   |   
Épisode terminé


## Évaluation

Afin de disposer d'un critère simple pour comparer les différentes solutions, nous utiliserons la valeur du portefeuille (`portfolio_valuation`).
C'est assez simple : on veut que l'agent ait gagné le plus d'argent à la fin de la simulation.

Vous pouvez ajouter ce critère à la liste des métriques affichées à la fin de chaque épisode, pour que ce soit plus visible :

In [10]:
def metric_portfolio_valuation(history):
    return round(history['portfolio_valuation', -1], 2)

env.add_metric('Portfolio Valuation', metric_portfolio_valuation)

done = False
obs, _ = env.reset()

while not done:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

Market Return :  5.27%   |   Portfolio Return : -100.00%   |   Portfolio Valuation : 0.0   |   


Puisque l'environnement peut se dérouler sur plusieurs épisodes (1 par jeu de données), vous devrez calculer la **moyenne des `portfolio_valuation`** sur l'ensemble des jeux de données possibles.

⚠️ Pour que ce soit honnête, vous **devez initialiser l'environnement avec les contraintes** imposées dans le sujet :

- une valeur initiale du portefeuille de `1000` ;
- des frais de 0.1% par transaction ;
- un taux d'intérêt de 0.02% par jour soit 0.02/100/24 par heure.

Sinon, il est beaucoup plus simple d'augmenter la valeur finale...

```py
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    # LIGNES SUIVANTES :
    # Valeur initiale du portefeuille
    portfolio_initial_value=1_000,
    # Frais de transactions
    trading_fees=0.1/100,
    # Intérêts sur les prêts
    borrow_interest_rate=0.02/100/24,
)
```

Vous pouvez également accéder à la métrique de `portfolio_valuation` à la fin d'une simulation, si vous voulez par exemple l'ajouter à votre *run* WandB :

In [11]:
portfolio_valuation = env.historical_info['portfolio_valuation', -1]
# Si on avait WandB :
# run.summary['portfolio_valuation'] = portfolio_valuation
# On simule ça par un simple print...
print(portfolio_valuation)

0.0039974649677703925


Ou bien, pour récupérer les métriques calculées par l'environnement (cela peut être utile pour les ajouter à WandB) :

In [12]:
metrics = env.get_metrics()
print(metrics)
portfolio_valuation = metrics['Portfolio Valuation']
print(portfolio_valuation)

{'Market Return': ' 5.27%', 'Portfolio Return': '-100.00%', 'Portfolio Valuation': np.float64(0.0)}
0.0


## Conseils

À part les quelques contraintes présentées dans ce fichier (et rappelées sur la page du projet), vous êtes assez libres !

Votre algorithme de RL peut être arbitrairement simple ou complexe. Je liste ici quelques conseils ou pistes, que vous pouvez explorer :

- *Features* : Par défaut, votre agent n'utilise que le prix de l'*asset* (`close`) comme *feature* pour la prise de décision. Vous pouvez ajouter les *features* que vous voulez. En particulier, des métriques spécifiques à la finance peuvent être intéressantes, par exemple pour déterminer le risque que le prix change brutalement (à la hausse ou à la baisse)...

- Algorithme : Vous pouvez utiliser des algorithmes existants, ou en inventer un nouveau. N'hésitez pas à ré-utiliser tout ce que vous avez appris en *Machine Learning* et *Deep Learning*. Typiquement, les données financières sont des données temporelles : certains réseaux de neurones sont plus appropriés que d'autres pour ce genre de tâche...

- Configuration de l'environnement : L'environnement est très extensible ! Vous pouvez par exemple ajouter des *features* dynamiques (pas seulement calculées lors du prétraitement). La [documentation](https://gym-trading-env.readthedocs.io/en/latest/index.html) est très claire et très complète.

Vous pouvez vous inspirer de travaux existants trouvés sur l'Internet à condition de **citer votre source**. Utiliser le travail de quelqu'un d'autre sans le citer sera considéré comme du plagiat.